<a href="https://colab.research.google.com/github/Krzesimir13/DataQuest/blob/main/eurgbp_visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization on EURGBP Exchange Rates


## 0. Preparation of the environment
Import of libraries and the dataset from MyDrive.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import kagglehub

In [2]:
path = kagglehub.dataset_download("lsind18/euro-exchange-daily-rates-19992020")
file = Path(path) / 'euro-daily-hist_1999_2022.csv'
data_raw = pd.read_csv(file)
data = data_raw.copy()

100%|██████████| 667k/667k [00:00<00:00, 82.6MB/s]

Extracting files...


## 1. Data preparation

### 1.1. Preview for the dataset

In [3]:
data.head()

,Period\Unit:,[Australian dollar ],[Bulgarian lev ],[Brazilian real ],[Canadian dollar ],[Swiss franc ],[Chinese yuan renminbi ],[Cypriot pound ],[Czech koruna ],[Danish krone ],...,[Romanian leu ],[Russian rouble ],[Swedish krona ],[Singapore dollar ],[Slovenian tolar ],[Slovak koruna ],[Thai baht ],[Turkish lira ],[US dollar ],[South African rand ]
0,2025-04-02,1.7146,1.9558,6.1212,1.5479,0.9543,7.8529,NaN,24.963,7.4611,...,4.9775,NaN,10.764,1.4508,NaN,NaN,36.93,40.9573,1.0803,20.1042
1,2025-04-01,1.7255,1.9558,6.1679,1.5529,0.952,7.8431,NaN,24.954,7.4616,...,4.9774,NaN,10.816,1.4492,NaN,NaN,36.846,40.9201,1.0788,19.7741
2,2025-03-31,1.7318,1.9558,6.2507,1.5533,0.9531,7.8442,NaN,24.962,7.4613,...,4.9771,NaN,10.849,1.4519,NaN,NaN,36.706,41.0399,1.0815,19.8782
3,2025-03-28,1.712,1.9558,6.2252,1.5444,0.9525,7.8445,NaN,24.96,7.4616,...,4.9774,NaN,10.82,1.4481,NaN,NaN,36.664,41.0387,1.0797,19.6113
4,2025-03-27,1.7101,1.9558,6.2154,1.5425,0.9524,7.8361,NaN,24.982,7.4605,...,4.9773,NaN,10.8235,1.445,NaN,NaN,36.529,40.9940,1.0785,19.7061


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6785 entries, 0 to 6784
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Period\Unit:              6785 non-null   object 
 1   [Australian dollar ]      6785 non-null   object 
 2   [Bulgarian lev ]          6383 non-null   object 
 3   [Brazilian real ]         6517 non-null   object 
 4   [Canadian dollar ]        6785 non-null   object 
 5   [Swiss franc ]            6785 non-null   object 
 6   [Chinese yuan renminbi ]  6517 non-null   object 
 7   [Cypriot pound ]          2346 non-null   object 
 8   [Czech koruna ]           6785 non-null   object 
 9   [Danish krone ]           6785 non-null   object 
 10  [Estonian kroon ]         3130 non-null   object 
 11  [UK pound sterling ]      6785 non-null   object 
 12  [Greek drachma ]          520 non-null    object 
 13  [Hong Kong dollar ]       6785 non-null   object 
 14  [Croatia

The dataset contains exchange rates of 40 currencies related to euro. I focus on EURGBP exchange rate towards years.


---

### 1.2. Preparing the target data

The new dataset object will contain only EURGBP rates. I will cut only EURGBP rates.

In [5]:
data.columns

Index(['Period\Unit:', '[Australian dollar ]', '[Bulgarian lev ]',
       '[Brazilian real ]', '[Canadian dollar ]', '[Swiss franc ]',
       '[Chinese yuan renminbi ]', '[Cypriot pound ]', '[Czech koruna ]',
       '[Danish krone ]', '[Estonian kroon ]', '[UK pound sterling ]',
       '[Greek drachma ]', '[Hong Kong dollar ]', '[Croatian kuna ]',
       '[Hungarian forint ]', '[Indonesian rupiah ]', '[Israeli shekel ]',
       '[Indian rupee ]', '[Iceland krona ]', '[Japanese yen ]',
       '[Korean won ]', '[Lithuanian litas ]', '[Latvian lats ]',
       '[Maltese lira ]', '[Mexican peso ]', '[Malaysian ringgit ]',
       '[Norwegian krone ]', '[New Zealand dollar ]', '[Philippine peso ]',
       '[Polish zloty ]', '[Romanian leu ]', '[Russian rouble ]',
       '[Swedish krona ]', '[Singapore dollar ]', '[Slovenian tolar ]',
       '[Slovak koruna ]', '[Thai baht ]', '[Turkish lira ]', '[US dollar ]',
       '[South African rand ]'],
      dtype='object')

In [29]:
eurgbp = data.loc[:,[r'Period\Unit:','[UK pound sterling ]']]
eurgbp = eurgbp.rename(columns={r'Period\Unit:' : 'Date',
                       '[UK pound sterling ]' : 'EURGBP'},)
eurgbp = eurgbp.replace('-', np.nan).dropna().sort_index()
eurgbp['EURGBP'] = eurgbp['EURGBP'].astype(float)
eurgbp['Date'] = pd.to_datetime(eurgbp['Date'])

In [30]:
eurgbp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6723 entries, 0 to 6784
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    6723 non-null   datetime64[ns]
 1   EURGBP  6723 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 157.6 KB


## 2. Data visualisation

I cut the data from 2010, to make the plot faster. And I add some rolling means.

In [51]:
eurgbp_all = eurgbp.copy()
eurgbp = eurgbp.loc[eurgbp['Date'] > '2010', :]
eurgbp['MA50'] = eurgbp['EURGBP'].rolling(50).mean()
eurgbp['MA100'] = eurgbp['EURGBP'].rolling(100).mean()
eurgbp['MA200'] = eurgbp['EURGBP'].rolling(200).mean()

Below, it is the template like in Financial Times.

In [47]:
# @title
import plotly.io as pio

ft_template = {
    "layout": {
        "font": {
            "family": "Arial",
            "size": 14,
            "color": "#333333"
        },

        # FT-like background
        "paper_bgcolor": "#f4f2e9",
        "plot_bgcolor": "#f4f2e9",

        # Axes
        "xaxis": {
            "showgrid": True,
            "gridcolor": "#d8d6cc",     # subtle grid
            "gridwidth": 0.7,
            "linecolor": "#333333",
            "linewidth": 1,
            "ticks": "outside",
            "zeroline": False
        },

        "yaxis": {
            "showgrid": True,
            "gridcolor": "#d8d6cc",
            "gridwidth": 0.7,
            "linecolor": "#333333",
            "linewidth": 1,
            "ticks": "outside",
            "zeroline": False
        },

        # Title centered, FT style
        "title": {
            "x": 0.5,
            "xanchor": "center",
            "font": {"size": 22, "color": "#111111"}
        },

        # Simple FT-like legend
        "legend": {
            "orientation": "h",
            "yanchor": "bottom",
            "y": 1.05,
            "xanchor": "right",
            "x": 1
        }
    }
}

pio.templates["ft"] = ft_template

In [95]:
fig = px.line(eurgbp, x='Date', y=['EURGBP', 'MA50', 'MA100', 'MA200'],
        title='EURGBP')

fig.update_layout(xaxis_title="Exchange rate", yaxis_title="Date",
                  template='ft')

fig.data[0].line.update(color="#2a2a2a", width=2)
fig.data[1].line.update(color="#d16a7c", width=1.6)
fig.data[2].line.update(color="#4b6ca8", width=1.6)
fig.data[3].line.update(color="#7a8f82", width=1.6)

brexit_vote = eurgbp.loc[eurgbp["Date"] == "2016-06-23"]
brexit_official = eurgbp.loc[eurgbp["Date"] == "2020-02-03"]

fig.add_scatter(
    x=brexit_vote['Date'],
    y=brexit_vote['EURGBP'],
    mode="markers+text",
    marker=dict(size=10, color="red"),
    text=["Brexit voting"],
    textposition="middle right",
    textfont=dict(size=14, color="red", family="Arial")
)

fig.add_scatter(
    x=brexit_official['Date'],
    y=brexit_official['EURGBP'],
    mode="markers+text",
    marker=dict(size=10, color="red"),
    text=["Brexit"],
    textposition="middle right",
    textfont=dict(size=14, color="red", family="Arial")
)

fig.add_shape(type="line",
    x0="2010-01-01", x1="2010-05-11",
    y0=0.67, y1=0.67,
    line=dict(color="pink", width=4)
)
fig.add_annotation(
    x="2010-03-01", y=0.68,
    text="<b>Gordon Brown</b>",
    showarrow=False,
    font=dict(size=14, color="black")
)

fig.add_shape(type="line",
    x0="2019-07-24", x1="2022-09-06",
    y0=0.67, y1=0.67,
    line=dict(color="red", width=4)
)
fig.add_annotation(
    x="2021-01-01", y=0.68,
    text="<b>Boris Johnson</b>",
    showarrow=False,
    font=dict(size=14, color="black")
)

fig.add_shape(type="line",
    x0="2016-07-13", x1="2019-07-24",
    y0=0.67, y1=0.67,
    line=dict(color="yellow", width=4)
)
fig.add_annotation(
    x="2017-12-31", y=0.68,
    text="<b>Theresa May</b>",
    showarrow=False,
    font=dict(size=14, color="red")
)


fig.show()

In [74]:
eurgbp.loc[eurgbp["Date"] == "2016-06-23"]['EURGBP']


,EURGBP
2248,0.76595
